In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model
from keras.optimizers import Adam
(X_train,y_train),(X_test,y_test)= tf.keras.datasets.cifar10.load_data()
print(type(X_train),type(y_train),type(X_test),type(y_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [30]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
no_of_datapoints_train,pixel_x_train,pixel_y_train,rgb_train=X_train.shape
no_of_datapoints_test,pixel_x_test,pixel_y_test,rgb_test=X_test.shape

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [31]:
# classes=["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
not_animal_classes=[0,1,8,9]
X_train_not_animal=[]
X_train_animal=[]
y_train_animal=[]
y_train_not_animal=[]
# print(y_train[0:10])
for i in range(no_of_datapoints_train):
  if(y_train[i][0] in not_animal_classes):
    y_train_not_animal.append(y_train[i])
    X_train_not_animal.append(X_train[i])
  else:
    X_train_animal.append(X_train[i])
    y_train_animal.append(y_train[i])
print(len(X_train_animal),len(y_train_animal))



30000 30000


In [32]:
X_test_not_animal=[]
X_test_animal=[]
y_test_animal=[]
y_test_not_animal=[]
# print(y_train[0:10])
for i in range(no_of_datapoints_test):
  if(y_test[i][0] in not_animal_classes):
    y_test_not_animal.append(y_test[i])
    X_test_not_animal.append(X_test[i])
  else:
    X_test_animal.append(X_test[i])
    y_test_animal.append(y_test[i])
print(len(X_test_animal),len(y_test_animal))


6000 6000


In [20]:
print(len(y_train_not_animal),len(X_train_not_animal))
X_train_animal = np.array(X_train_animal)
X_train_not_animal = np.array(X_train_not_animal)
y_train_animal = np.array(y_train_animal)
y_train_not_animal = np.array(y_train_not_animal)
print(X_train_animal.shape,X_train_not_animal.shape)
print(y_train_animal.shape, y_train_not_animal.shape)

20000 20000
(30000, 32, 32, 3) (20000, 32, 32, 3)
(30000, 1) (20000, 1)


In [33]:
print(len(y_test_not_animal),len(X_test_not_animal))
X_test_animal = np.array(X_test_animal)
X_test_not_animal = np.array(X_test_not_animal)
y_test_animal = np.array(y_test_animal)
y_test_not_animal = np.array(y_test_not_animal)
print(X_test_animal.shape,X_test_not_animal.shape)
print(y_test_animal.shape, y_test_not_animal.shape)

4000 4000
(6000, 32, 32, 3) (4000, 32, 32, 3)
(6000, 1) (4000, 1)


In [21]:
# Define the experts
def expert(input_shape):
    input_layer = Input(shape=input_shape)
    hidden_layer1 = Dense(3, activation='relu')(input_layer)
    hidden_layer2 = Dense(3, activation='relu')(hidden_layer1)
    output_layer = Dense(1, activation='sigmoid')(hidden_layer2)
    expert_model = Model(inputs=input_layer, outputs=output_layer)
    return expert_model

# Define the gating network
def gating(input_shape, num_experts):
    input_layer = Input(shape=input_shape)
    hidden_layer = Dense(num_experts, activation='softmax')(input_layer)
    gating_model = Model(inputs=input_layer, outputs=hidden_layer)
    return gating_model

In [24]:
num_experts = 5
ensemble_model_input = Input(shape=(32, 32, 3))
expert_outputs = [expert_model(ensemble_model_input) for _ in range(num_experts)]
gating_model = gating(input_shape=X_train_animal.shape[1:],num_experts=num_experts)
gating_output = gating_model(ensemble_model_input)
ensemble_output = Dense(1, activation='sigmoid')(gating_output * expert_outputs[0])
for i in range(1, num_experts):
    ensemble_output = ensemble_output + Dense(1, activation='sigmoid')(gating_output * expert_outputs[i])
ensemble_output = ensemble_output / num_experts
ensemble_model = Model(inputs=ensemble_model_input, outputs=ensemble_output)


In [27]:
ensemble_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [37]:
ensemble_model.fit(X_train_animal, y_train_animal, batch_size=128, epochs=50, validation_data=(X_test_animal, y_test_animal))

# Evaluate the ensemble model
test_loss, test_acc = ensemble_model.evaluate(X_test_animal, y_test_animal)
print('Test accuracy:', test_acc)

ValueError: ignored

In [36]:
# # Train the experts
# num_experts = 5
# experts = []
# for i in range(num_experts):
#     expert_model = expert(input_shape=X_train_animal.shape[1:])
#     expert_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     expert_model.fit(X_train_animal, (y_train_animal==i).astype(int), epochs=10, batch_size=32)
#     experts.append(expert_model)